In [ ]:
import os
import napari
import numpy as np
import tifffile
from skimage import measure
from napari_animation import Animation

parent_folder = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220511_RM0008_126hpf_fP10_f2\final_alignment'

# Load data
em_stack = tifffile.imread(os.path.join(parent_folder, 'fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15.tif'))
em_mask = tifffile.imread(os.path.join(parent_folder, 'fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15_cp_masks_ft00_cp-3.tif'))
lm_stack = tifffile.imread(os.path.join(parent_folder, 'flipped_upsampled_clahe_gaussian_1_20220511_RM0008_126hpf_fP10_f2_anatomyGFRF_001_.tif'))
em_warped_stack = tifffile.imread(os.path.join(parent_folder, 'fine_aligned_em_stack_20220511_RM0008_126hpf_fP10_f2_ds4_woResin_144nm_px_from15_resLM_fovLM.tif'))

In [4]:
lm_stack.shape

(375, 2, 512, 512)

In [12]:

# Create viewer
viewer = napari.Viewer(ndisplay=2)

# Add layers with translations
#em_stack_layer = viewer.add_image(em_stack, name='EM Stack', colormap='gray', visible=True)
#em_mask_layer = viewer.add_labels(em_mask, name='EM Mask', visible=True)

lm_stack_layer_g = viewer.add_image(lm_stack[:,0], name='LM Stack Green', colormap='green', visible=True)
lm_stack_layer_r = viewer.add_image(lm_stack[:,1], name='LM Stack Red', colormap='red', visible=True)

em_warped_stack_layer = viewer.add_image(em_warped_stack, name='EM Warped Stack', colormap='gray', visible=True)

D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


In [14]:
# Adjust viewer size to be divisible by 16
viewer.window.resize(896, 896)

# Create animation
animation = Animation(viewer)

# 1. EM stack scrolling
animation.capture_keyframe()
for i in range(0, 50, 2):  # Increased step size
    viewer.dims.set_point(0, i)
    animation.capture_keyframe()
    
# 3. LM stack at the comparison frame + first centroid
lm_stack_layer_g.visible = False
lm_stack_layer_r.visible = False
animation.capture_keyframe(steps=2)
lm_stack_layer_g.visible = True
lm_stack_layer_r.visible = True
animation.capture_keyframe(steps=2)  # Reduced steps
lm_stack_layer_g.visible = False
lm_stack_layer_r.visible = False
animation.capture_keyframe(steps=2)
lm_stack_layer_g.visible = True
lm_stack_layer_r.visible = True
animation.capture_keyframe(steps=2) 

for i in range(em_stack.shape[0]//2,em_stack.shape[0], 10):  # Increased step size
    viewer.dims.set_point(0, i)
    animation.capture_keyframe()

# Save animation
animation.animate('clem_alignment_sequence2.mp4', 
                 fps=30,
                 canvas_only=True )   # Increased fps

napari.run()

Rendering frames...


100%|██████████| 954/954 [00:54<00:00, 17.38it/s]


In [16]:
# Create viewer with specific size first
viewer = napari.Viewer(ndisplay=2)
viewer.window.resize(1104, 1648)  # Multiple of 16 that matches your image dimensions

# Add layers
lm_stack_layer_g = viewer.add_image(lm_stack[:,0], name='LM Stack Green', colormap='green', visible=True)
lm_stack_layer_r = viewer.add_image(lm_stack[:,1], name='LM Stack Red', colormap='red', visible=True)
em_warped_stack_layer = viewer.add_image(em_warped_stack, name='EM Warped Stack', colormap='gray', visible=True)

# Create animation
animation = Animation(viewer)
viewer.dims.set_point(0, 0)
## Modified animation sequence
# 1. Initial state
animation.capture_keyframe()

# 2. Scroll through EM stack first half
for i in range(0, 100, 5):  # Smaller step size for smoother scrolling
    viewer.dims.set_point(0, i)
    animation.capture_keyframe()

# 3. Show/Hide LM channels with longer transitions
# First flash
lm_stack_layer_g.visible = False
lm_stack_layer_r.visible = False
animation.capture_keyframe(steps=10)

lm_stack_layer_g.visible = True
lm_stack_layer_r.visible = True
animation.capture_keyframe(steps=10)  # Longer transition
lm_stack_layer_g.visible = False
lm_stack_layer_r.visible = False
animation.capture_keyframe(steps=10)

# Second flash
lm_stack_layer_g.visible = True
lm_stack_layer_r.visible = True
animation.capture_keyframe(steps=10)

# 4. Continue scrolling through second half
for i in range(100, em_warped_stack.shape[0], 5):
    viewer.dims.set_point(0, i)
    animation.capture_keyframe()

# Save animation with adjusted settings
animation.animate('clem_alignment_sequence3.mp4', 
                 fps=15,  # Lower fps for smoother playback
                 canvas_only=True)

napari.run()



D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


Rendering frames...


100%|██████████| 1166/1166 [02:45<00:00,  7.03it/s]
